# Biodiversity for food and agriculture - Dec 2017

The notebook assesses the contribution of biodiversity for food and agriculture through an analysis of overlay between IUCN's Red List of Threatened Species and the World Database on Protected Areas

## Understanding the data

In [1]:
import numpy as np
import pandas as pd

In [2]:
sis = pd.read_csv('sis_2017.csv')

In [5]:
bfa = pd.read_csv('Food_FAO_2017_2.csv', delimiter=';')

In [6]:
sis.head()

,objectid,id_no,binomial,presence,origin,seasonal,compiler,year,citation,source,...,tax_comm,kingdom,phylum,class,order_,family,genus,code,shape_length,shape_area
0,10143,190868.0,Nihonogomphus thomassoni,2,1,1,Kate Saunders,2011,"Asia freshwater biodiversity assessments, IUCN",Red List assessment,...,NaN,ANIMALIA,ARTHROPODA,INSECTA,ODONATA,GOMPHIDAE,Nihonogomphus,LC,99.744070,27.275351
1,10144,190868.0,Nihonogomphus thomassoni,1,1,1,FBU,2011,IUCN (International Union for Conservation of ...,NaN,...,NaN,ANIMALIA,ARTHROPODA,INSECTA,ODONATA,GOMPHIDAE,Nihonogomphus,LC,2.133768,0.093543
2,10145,190868.0,Nihonogomphus thomassoni,1,1,1,DO Manh Cuong,2010,"Asia freshwater biodiversity assessments, IUCN",unknown,...,NaN,ANIMALIA,ARTHROPODA,INSECTA,ODONATA,GOMPHIDAE,Nihonogomphus,LC,4.357552,0.301940
3,10146,190868.0,Nihonogomphus thomassoni,1,1,1,DO Manh Cuong,2010,"Asia freshwater biodiversity assessments, IUCN",Red List assessment,...,NaN,ANIMALIA,ARTHROPODA,INSECTA,ODONATA,GOMPHIDAE,Nihonogomphus,LC,6.100119,0.527099
4,10147,63940.0,Tantilla coronadoi,1,1,1,NatureServe and IUCN,2007,NatureServe and IUCN (International Union for ...,NaN,...,NaN,ANIMALIA,CHORDATA,REPTILIA,SQUAMATA,COLUBRIDAE,Tantilla,LC,0.789530,0.049604


In [13]:
print('The number of database rows of species id 137: {}'.format(sis[sis.id_no==137].id_no.count()))

The number of database rows of species id 137: 13


It is important to understand that the `sis` table, i.e., the attribute table of the spatial data, contain potentially both multi-part polygons as well as single-part polygons (probably due to different sources + range polygons with different presence, origin and seasonality). 

There may also be a possiblity that these rows may be overlapping.

Setting the indices for both datasets. Make sure the ids are integer. Also clean the tables, to reduce size

In [35]:
sis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78042 entries, 0 to 78041
Data columns (total 25 columns):
objectid        78042 non-null int64
id_no           78042 non-null float64
binomial        78042 non-null object
presence        78042 non-null int64
origin          78042 non-null int64
seasonal        78042 non-null int64
compiler        76165 non-null object
year            78042 non-null int64
citation        78038 non-null object
source          24156 non-null object
dist_comm       947 non-null object
island          7320 non-null object
subspecies      1197 non-null object
subpop          186 non-null object
legend          78042 non-null object
tax_comm        426 non-null object
kingdom         78042 non-null object
phylum          78042 non-null object
class           78042 non-null object
order_          78042 non-null object
family          78042 non-null object
genus           78042 non-null object
code            78042 non-null object
shape_length    78042 non-nul

In [37]:
sis.id_no.max()

117582065.0

Choose 4 bytes for the ID field. The largest `id_no` is less than the largest value of `uint32`

In [38]:
int_types = ["uint8", "int8", "int16", "uint16", "uint32", "int32", 'int64']
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for uint16
---------------------------------------------------------------
min = 0
max = 65535
---------------------------------------------------------------

Machine parameters for uint32
---------------------------------------------------------------
min = 0
max = 4294967295
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -

In [40]:
columns=['id_no', 'kingdom', 'phylum', 'class', 'order_', 'family', 'genus', 'code']
sis_clean = pd.DataFrame(sis[columns])
sis_clean = sis_clean.astype(dtype={"id_no": "uint32",
                                    'kingdom': "category",
                                    'phylum': "category",
                                     'class': "category",
                                     'order_': "category",
                                     'family': "category",
                                     'genus': "category",
                                     'code': "category"})
sis_clean = sis_clean.set_index('id_no')

In [41]:
sis_clean.info()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 78042 entries, 190868 to 87531801
Data columns (total 7 columns):
kingdom    78042 non-null category
phylum     78042 non-null category
class      78042 non-null category
order_     78042 non-null category
family     78042 non-null category
genus      78042 non-null category
code       78042 non-null category
dtypes: category(7)
memory usage: 1.8 MB


After the data cleaning process, the size of the `sis` table has reduced from 14.9+MB to 1.8MB. All essential information is retained and the number of rows stays the same

In [43]:
sis.index.size == sis_clean.index.size

True

In [20]:
sis.id_no.max()

117582065.0

In [7]:
bfa.head()

,id,kingdom_name,phylum_name,class_name,order_name,family_name,genus_name,species_name,scientific_name,authority,infra_rank,infra_name,infra_authority,category,criteria,publicationyear,main_common_name,value,REF,DESCRIPTION
0,9,ANIMALIA,CHORDATA,ACTINOPTERYGII,CYPRINIFORMES,CYPRINIDAE,Aaptosyax,grypus,Aaptosyax grypus,"Rainboth, 1991",NaN,NaN,NaN,CR,A2acd,2011,Mekong Giant Salmon Carp,1,1,Food - human
1,137,ANIMALIA,CHORDATA,MAMMALIA,CHIROPTERA,PTEROPODIDAE,Acerodon,celebensis,Acerodon celebensis,"Peters, 1867",NaN,NaN,NaN,VU,A2d,2016,Sulawesi Fruit Bat,1,1,Food - human
2,138,ANIMALIA,CHORDATA,MAMMALIA,CHIROPTERA,PTEROPODIDAE,Acerodon,humilis,Acerodon humilis,"K. Andersen, 1909",NaN,NaN,NaN,EN,"B1ab(iii,v)",2016,Talaud Fruit Bat,1,1,Food - human
3,139,ANIMALIA,CHORDATA,MAMMALIA,CHIROPTERA,PTEROPODIDAE,Acerodon,jubatus,Acerodon jubatus,"(Eschscholtz, 1831)",NaN,NaN,NaN,EN,A2cd,2016,Golden-capped Fruit Bat,1,1,Food - human
4,140,ANIMALIA,CHORDATA,MAMMALIA,CHIROPTERA,PTEROPODIDAE,Acerodon,leucotis,Acerodon leucotis,"(Sanborn, 1950)",NaN,NaN,NaN,VU,A4cd,2008,Palawan Fruit Bat,1,1,Food - human


In [33]:
bfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11396 entries, 0 to 11395
Data columns (total 20 columns):
id                  11396 non-null int64
kingdom_name        11396 non-null object
phylum_name         11396 non-null object
class_name          11396 non-null object
order_name          11396 non-null object
family_name         11396 non-null object
genus_name          11396 non-null object
species_name        11396 non-null object
scientific_name     11396 non-null object
authority           11380 non-null object
infra_rank          126 non-null object
infra_name          126 non-null object
infra_authority     0 non-null float64
category            11396 non-null object
criteria            1899 non-null object
publicationyear     11396 non-null int64
main_common_name    8649 non-null object
value               11396 non-null int64
REF                 11396 non-null int64
DESCRIPTION         11396 non-null object
dtypes: float64(1), int64(4), object(15)
memory usage: 1.7+ MB


In [44]:
bfa.id.max()

117582065

In [51]:
columns=['id', 'kingdom_name', 'phylum_name', 'class_name', 'order_name', 'family_name', 'genus_name', 'category']
bfa_clean = pd.DataFrame(bfa[columns])
bfa_clean = bfa_clean.astype(dtype={"id": "uint32",
                                    'kingdom_name': "category",
                                    'phylum_name': "category",
                                     'class_name': "category",
                                     'order_name': "category",
                                     'family_name': "category",
                                     'genus_name': "category",
                                     'category': "category"})
bfa_clean = bfa_clean.set_index('id')

In [52]:
bfa_clean.info()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 11396 entries, 9 to 117582065
Data columns (total 7 columns):
kingdom_name    11396 non-null category
phylum_name     11396 non-null category
class_name      11396 non-null category
order_name      11396 non-null category
family_name     11396 non-null category
genus_name      11396 non-null category
category        11396 non-null category
dtypes: category(7)
memory usage: 338.8 KB
